# Naive Bayesian Classifier
### Q1. Bayes Rule을 이해하고 Naive  Bayes classifier가 사용하는 사후 확률 계산 과정을 서술하세요.

- Bayes Rule:   
$P(w_i|x) = \frac{P(x|w_i)|P(w_i)}{P(x)} = \frac{P(x|w_i) P(w_i)}{\Sigma_j P(x|w_j)P(w_j)}$
  -
  - $P(x|w_i)\text{: 사후 확률, posterior}\\
P(x|w_i) \text{: 가능도/우도, likelihood}\\
P(w_i) \text{: 사전 확률, prior}\\
P(x) \text{: 증거, evidence}$

A1. 사후 확률을 계산할 때, 속성의 수가 증가할수록 특정 증거에 대한 데이터가 충분하지 않기 때문에 훈련이 충분이 되지 못한다. 그래서 Naive Bayes classifier는 각 특성이 각각 독립적으로 발생할 것이라는 나이브한 가정을 두고, 가능도를 각 특성에 대한 조건부 확률을 곱해서 구한다. 이후 사후 확률은 사전확률과 가능도를 곱하고 증거를 나눠서 구한다.

### Q2. Naive Bayes Classification 방법을 이용해서 다음 생성된 리뷰 데이터에 기반한 감정 분석을 해봅시다.

In [64]:
# pip install pandas
import pandas as pd
import re

In [65]:
# 리뷰 데이터 생성
data = {
    'review': [
        'I love this great product! It exceeded my expectations.',
        'The Worst purchase I have ever made. Completely useless.',
        'It is an average product, nothing special but not terrible either.',
        'Great service and who can help but love this design? Highly recommend!',
        'Terrible experience, I will never buy from this poor brand again.',
        'It’s acceptable, but I expected better service, not just an acceptable one.',
        'Absolutely wonderful! I am very satisfied with this great service.',
        'The quality is poor and it broke after one use. Terrible enough!',
        'Acceptable product for the price, but there are better options out there.',
        'Great quality and fast shipping with wonderful service! I love it'
    ],
    'sentiment': [
        'positive', 'negative', 'neutral', 'positive', 'negative',
        'neutral', 'positive', 'negative', 'neutral', 'positive',
    ]
}
df = pd.DataFrame(data)
df.head()

,review,sentiment
0,I love this great product! It exceeded my expe...,positive
1,The Worst purchase I have ever made. Completel...,negative
2,"It is an average product, nothing special but ...",neutral
3,Great service and who can help but love this d...,positive
4,"Terrible experience, I will never buy from thi...",negative


In [66]:
# 불용어 리스트 정의
stopwords = ['i', 'my', 'am', 'this', 'it', 'its', 'an', 'a', 'the', 'is', 'are', 'and', 'product', 'service']

In [67]:
# 텍스트 전처리 함수 정의
def preprocess_text(text):
    # 소문자로 변환
    text = text.lower()
    # 특수 기호 제거
    text = re.sub(r'[^a-z\s]', '', text)
    # 불용어 제거
    words = text.split()
    filtered_words = [word for word in words if word not in stopwords]
    return ' '.join(filtered_words)

# 모든 리뷰에 대해 전처리 수행
df['review'] = df['review'].apply(preprocess_text)

기본적인 데이터 전처리가 완료되었습니다!
이제부터 직접 나이브 베이지안 분류를 수행해 봅시다.  
우리가 분류하고자 하는 문장은 총 두가지 입니다.  
전처리가 완료되었다고 치고,   
첫번째 문장은 **'love, great, awesome'**,  
두번째 문장은 **'terrible, not, never'** 입니다.

사전 확률 $P(positive), P(negative), P(neutral)$을 구합니다.

In [68]:
# 사전 확률 구하는 코드를 작성해주세요.

value_counts = df['sentiment'].value_counts()
prior_probabilities = value_counts / len(df)

print(prior_probabilities)

sentiment
positive    0.4
negative    0.3
neutral     0.3
Name: count, dtype: float64


가능도를 구하기 위한 확률들을 계산합니다.  
예: 첫번째 문장 분류를 위해서는, $P(love|positive), P(great|positive), P(awesome|positive)\\
P(love|negative), P(great|negative), P(awesome|negative)\\
P(love|neutral), P(great|neutral), P(great|neutral)$를 구합니다.

이 때 CountVectorizer를 사용하여 도출한 단어 벡터를 활용하면 확률들을 간편하게 구할 수 있습니다.  
참고: https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html

In [69]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
review_array = vectorizer.fit_transform(df['review']).toarray()
review_array

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0,
        0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
        0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
        0, 0, 0, 1, 0, 0, 0, 0, 0, 1],
       [0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
        1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
        0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
       [0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
        1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
        1, 0, 0, 0, 0, 0, 1, 0, 0, 0],
       [0, 2, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 

In [70]:
vectorizer.get_feature_names_out()

array(['absolutely', 'acceptable', 'after', 'again', 'average', 'better',
       'brand', 'broke', 'but', 'buy', 'can', 'completely', 'design',
       'either', 'enough', 'ever', 'exceeded', 'expectations', 'expected',
       'experience', 'fast', 'for', 'from', 'great', 'have', 'help',
       'highly', 'just', 'love', 'made', 'never', 'not', 'nothing', 'one',
       'options', 'out', 'poor', 'price', 'purchase', 'quality',
       'recommend', 'satisfied', 'shipping', 'special', 'terrible',
       'there', 'use', 'useless', 'very', 'who', 'will', 'with',
       'wonderful', 'worst'], dtype=object)

In [71]:
vectorizer.vocabulary_

{'love': 28,
 'great': 23,
 'exceeded': 16,
 'expectations': 17,
 'worst': 53,
 'purchase': 38,
 'have': 24,
 'ever': 15,
 'made': 29,
 'completely': 11,
 'useless': 47,
 'average': 4,
 'nothing': 32,
 'special': 43,
 'but': 8,
 'not': 31,
 'terrible': 44,
 'either': 13,
 'who': 49,
 'can': 10,
 'help': 25,
 'design': 12,
 'highly': 26,
 'recommend': 40,
 'experience': 19,
 'will': 50,
 'never': 30,
 'buy': 9,
 'from': 22,
 'poor': 36,
 'brand': 6,
 'again': 3,
 'acceptable': 1,
 'expected': 18,
 'better': 5,
 'just': 27,
 'one': 33,
 'absolutely': 0,
 'wonderful': 52,
 'very': 48,
 'satisfied': 41,
 'with': 51,
 'quality': 39,
 'broke': 7,
 'after': 2,
 'use': 46,
 'enough': 14,
 'for': 21,
 'price': 37,
 'there': 45,
 'options': 34,
 'out': 35,
 'fast': 20,
 'shipping': 42}

In [72]:
frequency_matrix = pd.DataFrame(review_array, columns = vectorizer.get_feature_names_out())
frequency_matrix = pd.concat([df['sentiment'], frequency_matrix], axis=1)
frequency_matrix

,sentiment,absolutely,acceptable,after,again,average,better,brand,broke,but,...,terrible,there,use,useless,very,who,will,with,wonderful,worst
0,positive,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,negative,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,1
2,neutral,0,0,0,0,1,0,0,0,1,...,1,0,0,0,0,0,0,0,0,0
3,positive,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,1,0,0,0,0
4,negative,0,0,0,1,0,0,1,0,0,...,1,0,0,0,0,0,1,0,0,0
5,neutral,0,2,0,0,0,1,0,0,1,...,0,0,0,0,0,0,0,0,0,0
6,positive,1,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,1,1,0
7,negative,0,0,1,0,0,0,0,1,0,...,1,0,1,0,0,0,0,0,0,0
8,neutral,0,1,0,0,0,1,0,0,1,...,0,2,0,0,0,0,0,0,0,0
9,positive,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,1,0


In [73]:
# 위와 같이 조건부 확률을 구하는 코드를 작성해주세요

sentence1 = ['love', 'great', 'awesome']
sentence2 = ['terrible', 'not', 'never']


def conditional_probability(word, sentiment):

    vocab_size = 3 # 관심 있는 단어 리스트의 길이
    frequency_matrix_sen = frequency_matrix[frequency_matrix['sentiment'] == sentiment] # 해당 감정만 필터링
    total_words = frequency_matrix_sen.drop(columns='sentiment').sum().sum() # 총 단어 수

    if word in frequency_matrix_sen.columns: # 문서에 단어가 존재하는 경우
        prob_word_given_sen = (frequency_matrix_sen[word].sum() + 1) / (total_words + vocab_size)
    else:
        prob_word_given_sen = 1 / (total_words + vocab_size)

    return prob_word_given_sen

print('target_review1')
for sentiment in ['positive', 'negative', 'neutral']:
  for word in sentence1:
    print(f"P({word}|{sentiment})':{conditional_probability(word, sentiment):.10f}")

print(" ")

print('target_review2')
for sentiment in ['positive', 'negative', 'neutral']:
  for word in sentence2:
    print(f"P({word}|{sentiment})':{conditional_probability(word, sentiment):.10f}")

target_review1
P(love|positive)':0.1379310345
P(great|positive)':0.1724137931
P(awesome|positive)':0.0344827586
P(love|negative)':0.0370370370
P(great|negative)':0.0370370370
P(awesome|negative)':0.0370370370
P(love|neutral)':0.0370370370
P(great|neutral)':0.0370370370
P(awesome|neutral)':0.0370370370
 
target_review2
P(terrible|positive)':0.0344827586
P(not|positive)':0.0344827586
P(never|positive)':0.0344827586
P(terrible|negative)':0.1111111111
P(not|negative)':0.0370370370
P(never|negative)':0.0740740741
P(terrible|neutral)':0.0740740741
P(not|neutral)':0.1111111111
P(never|neutral)':0.0370370370


독립성 가정을 이용하여 가능도(likelihood)를 구합니다.  
첫번째 문장 예시: $P(love, great, awesome|positive), P(love, great, awesome|negative), P(love, great, awesome|neutral)$

In [74]:
# 가능도 구하는 코드를 작성해주세요.

def likelihood(sentence, sentiment):
    likelihood = 1
    for word in sentence:
        likelihood *= conditional_probability(word, sentiment)
    return likelihood

print('target_review1')
for sentiment in ['positive', 'negative', 'neutral']:
    likelihood_value = likelihood(sentence1, sentiment)
    print(f"Likelihoods for '{sentiment}': {likelihood_value:.10f}")

print(" ")

print('target_review2')
for sentiment in ['positive', 'negative', 'neutral']:
    likelihood_value = likelihood(sentence2, sentiment)
    print(f"Likelihoods for '{sentiment}': {likelihood_value:.10f}")

target_review1
Likelihoods for 'positive': 0.0008200418
Likelihoods for 'negative': 0.0000508053
Likelihoods for 'neutral': 0.0000508053
 
target_review2
Likelihoods for 'positive': 0.0000410021
Likelihoods for 'negative': 0.0003048316
Likelihoods for 'neutral': 0.0003048316


위에서 구한 사전 확률과 가능도를 이용하여 타겟 문장이 positive, negative, neutral일 확률을 구하고 최종적으로 어떤 감성일지 분석해봅니다.

In [75]:
import numpy as np
# 최종 확률 구하는 코드를 작성해주세요.

# 사전 확률
prior_positive = 0.4
prior_negative = 0.3
prior_neutral = 0.3

# 첫번째 문장

# 가능도
likelihood_positive_1 = likelihood(sentence1, 'positive')
likelihood_negative_1 = likelihood(sentence1, 'negative')
likelihood_neutral_1 = likelihood(sentence1, 'neutral')

# 사후 확률 계산
posterior_positive_1 = likelihood_positive_1 * prior_positive
posterior_negative_1 = likelihood_negative_1 * prior_negative
posterior_neutral_1 = likelihood_neutral_1 * prior_neutral

print(f"P(positive|target_review1) = {posterior_positive_1:.10f}") # P(positive|target_review1)
print(f"P(negative|target_review1) = {posterior_negative_1:.10f}") # P(negative|target_review1)
print(f"P(neutral|target_review1) = {posterior_neutral_1:.10f}") # P(neutral|target_review1)

print("")
# 두번째 문장

# 가능도
likelihood_positive_2 = likelihood(sentence2, 'positive')
likelihood_negative_2 = likelihood(sentence2, 'negative')
likelihood_neutral_2 = likelihood(sentence2, 'neutral')

# 사후 확률 계산
posterior_positive_2 = likelihood_positive_2 * prior_positive
posterior_negative_2 = likelihood_negative_2 * prior_negative
posterior_neutral_2 = likelihood_neutral_2 * prior_neutral

print(f"P(positive|target_review2) = {posterior_positive_2:.10f}") # P(positive|target_review2)
print(f"P(negative|target_review2) = {posterior_negative_2:.10f}") # P(negative|target_review2)
print(f"P(neutral|target_review2) = {posterior_neutral_2:.10f}") # P(neutral|target_review2)

P(positive|target_review1) = 0.0003280167
P(negative|target_review1) = 0.0000152416
P(neutral|target_review1) = 0.0000152416

P(positive|target_review2) = 0.0000164008
P(negative|target_review2) = 0.0000914495
P(neutral|target_review2) = 0.0000914495


A2-1.   
Target review1의 분류 결과: positive

Target review2의 분류 결과: negative or neutral

Q2-2. 나이브 베이지안 기반 확률을 구하는 과정에서 어떤 문제점을 발견할 수 있었나요? 그리고 그 문제를 해결하기 위한 방법에 대해 간략하게 조사 및 서술해 주세요. (힌트: Laplace smoothing)

A2-2.
조건부 확률을 구하는 함수를 작성하는 과정에서 'awesome'이 'positive'인 문장에 존재하지 않아 KeyError가 발생하였다. 이 문제를 해결하기 위해서 Laplace smoothing 방법을 사용하였다. 이는 확률을 계산하는 과정에서 발생할 수 있는 0 확률 문제를 해결하기 위한 방법 중 하나이다. 자연어 처리와 같은 분야에서 텍스트 데이터를 다룰 때, 특정 단어가 데이터에서 전혀 등장하지 않을 때 발생하는 문제를 해결하기 위해 많이 사용된다. 각 클래스에서 각 특징의 발생 확률을 계산할 때, 해당 특징이 한 번도 발생하지 않아 0이 되는 경우를 방지한다.